In [2]:
import tensorflow as tf
import tensorflow_addons as tfa
import tensorflow_text as text
import tensorflow_hub as hub

In [4]:
import pandas as pd

In [5]:
df = pd.read_pickle('../data/prepared/train.pkl')

In [8]:
df['positive'].head().tolist()

['Недавно устроился в ****** на должность кладовщика. Коллектив дружный и приветливый к новичкам - во всем помогают и поддерживают. Зарплата вся белая и без задержек, что сейчас редкость. Всем доволен и рекомендую данного работодателя.',
 'Расположение офиса, своего рода стабильность (госбанк)',
 'В нашем банке все по закону в плане зарплаты и графика. Никаких переработок и прочего. Для меня второе очень важно потому что у меня дети , с ними тоже надо проводить время, а если я буду до 12 засиживаться в офисе, то не знаю как и управиться с ними) Короче, работой в этом банке я довольна',
 'Я в\xa0****** около года работаю оператором. Нормальная работа, коллектив у\xa0нас дружный. Начальство понимающее. А\xa0самое главное, что как обещали на\xa0собеседовании, так и\xa0платят. Все честно. Я\xa0доволен.',
 'В принципе норм.Работать можно']

In [3]:
preprocessor = hub.load("https://tfhub.dev/jeongukjae/distilbert_multi_cased_preprocess/2")
tokenize = hub.KerasLayer(preprocessor.tokenize)
bert_pack_inputs = hub.KerasLayer(preprocessor.bert_pack_inputs)
encoder = hub.KerasLayer("https://tfhub.dev/jeongukjae/distilbert_multi_cased_L-6_H-768_A-12/1", trainable=True)

text_inputs = [
    tf.keras.layers.Input(shape=(), dtype=tf.string, name='1'),
    tf.keras.layers.Input(shape=(), dtype=tf.string, name='2'),
]
tokenized_inputs = [tokenize(segment) for segment in text_inputs]
encoder_inputs = bert_pack_inputs(tokenized_inputs)
encoder_outputs = encoder(encoder_inputs)

pooled_output = encoder_outputs["pooled_output"]      # [batch_size, 768].
sequence_output = encoder_outputs["sequence_output"]  # [batch_size, seq_length, 768].

model = tf.keras.Model(text_inputs, pooled_output)

# You can pass your sentences as follows
hypotheses = tf.constant(["(your hypothesis text here)"])
premises = tf.constant(["(your premise text here)"])
print(model([hypotheses, premises]))

2022-01-23 18:08:41.931835: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-01-23 18:08:41.939129: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-01-23 18:08:41.939538: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-01-23 18:08:41.940244: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

tf.Tensor(
[[-9.77929384e-02 -1.90723017e-02  2.87879735e-01  2.01715261e-01
   1.32675678e-01  1.87418595e-01  7.09430277e-02 -4.32238579e-02
   7.86442086e-02  8.48151445e-02  2.33148038e-01 -4.09674868e-02
   1.42586172e-01  2.06782371e-02 -5.30940890e-01 -8.82382505e-03
  -2.30553806e-01  3.85083944e-01 -6.20651692e-02  1.33203790e-01
   1.63899645e-01  3.94610241e-02 -1.48755372e-01  4.73004729e-02
   5.06999791e-02 -4.74096835e-01  1.19097643e-01  2.44544655e-01
   3.56038511e-01  1.63824379e-01  3.35829891e-02  3.22408289e-01
   1.12293512e-02  3.03551495e-01  1.58607215e-02  3.87430303e-02
  -1.87309861e+00 -1.53507397e-01 -1.43844634e-01 -1.11902818e-01
   9.11732092e-02 -2.26153523e-01  1.33510143e-01  1.98240466e-02
   2.85201296e-02  1.10115111e+00  3.52883935e-02  2.96758004e-02
   1.00071466e+00  2.28553116e-02  1.84893571e-02 -6.27210617e-01
   1.95877224e-01 -1.30717015e+00  1.71929806e-01  1.92422152e-01
   5.63004725e-02 -1.54985771e-01 -6.60677105e-02  2.28826419e-01